# AWS ML: Translate
In the previous walkthrough, we looked at a machine learning tool for image and video analysis called Amazon Rekognition. In this walkthrough, we will look at a machine learning service called Amazon Translate. The [documentation](https://docs.aws.amazon.com/translate/latest/dg/what-is.html) says that this tool is a: 
>*a neural machine translation service for translating text to and from English across a breadth of supported languages....You can use Amazon Translate to translate unstructured text documents or to build applications that work in multiple languages.*

Below, I will demonstrate how to call the service from within a Sagemaker notebook using AWS CLI (bash) and Python SDK (using the boto 3 package)

## Setting up an IAM Role

To use this service within SageMaker, we will first need to update the Role in the SageMaker permissions, just like we did for the Rekognition service. 

### Finding the Role

Begin by locating the SageMaker dashboard after loading the Learner Lab through AWS Academy Canvas page. Go to *Notebook Instances* under the "Notebook" tab on the left of the dashboard. Click on the name of the notebook instance you will be using to access the service. Just like in the Rekognition notebook, you will end up at a page that looks similar to the one below: 

![Notebook instance settings](./notebook-instance-screenshot) 

Under the "Permissions and encryption" heading, click the link to the IAM role ARN. The page below will appear. 

![IAM_role](./IAM-arn-role-screenshot)

### Adding a new policy

I already added the `TranslateFullAccess` policy that is needed for this walkthrough, but the policy can be added by going to the IAM summary page and clicking the blue *Attach policies* button. This is exactly how it is done in the Rekognition notebook, so I will not go more in-depth here. 

## Programming to Translate Text: Two Approaches

In continuation of the walkthrough, we will run commands that can translate text using two methods: 1) using the AWS CLI and 2) using AWS SDK for Python. If you would like to view more information on how exactly the machine learning service works, you can access that documentation [here](https://docs.aws.amazon.com/translate/latest/dg/how-it-works.html). 

### AWS CLI Method

Begin by opening up a `conda_python3` notebook in a new folder in the Jupyter Lab. The creation of this folder will allow you to keep everything organized, specifically the .json file you will create that contains the text you will translate. 

You will begin by running the following command to check that the Translate service is running. 

In [35]:
!aws translate help

TRANSLATE()                                                        TRANSLATE()



NAME
       translate -

DESCRIPTION
       Provides  translation  between  one  source language and another of the
       same set of languages.

AVAILABLE COMMANDS
       +o create-parallel-data

       +o delete-parallel-data

       +o delete-terminology

       +o describe-text-translation-job

       +o get-parallel-data

       +o get-terminology

       +o help

       +o import-terminology

       +o list-parallel-data

       +o list-terminologies

       +o list-text-translation-jobs

       +o start-text-translation-job

       +o stop-text-translation-job

       +o translate-text

       +o update-parallel-data



                                                                   TRANSLATE()


If you added the policy, this command should work. 

We will now use the `translate-text` command within this service to translate text from a .json into another .json. 

For this example, I will be translating an excerpt from Fyodor Dostoevsky's *The Beggar Boy at Christ’s Christmas Tree* (1876), a short story in the Russian language that can be found [here](https://azbyka.ru/fiction/malchik-u-xrista-na-elke/) under Section 2. 

#### An Important Note

Unfortunately, I cannot translate the entire story because the `translate-text` command can only translate 5,000 bytes at a time. Therefore, for the purposes of this walkthrough, I have chosen a smaller portion of this short story to translate. 

#### Back to it... 

To create the .json that we will translate, navigate to a [.json editor](https://jsoneditoronline.org/#left=local.yixifa). 

Using the example format below: 

In [36]:
{
    "Text": "Amazon Translate translates documents between languages in   real time. It uses advanced machine learning technologies 
    to provide high-quality real-time translation. Use it to 
    translate documents or to build applications that work in 
    multiple languages.", 
    "SourceLanguageCode": "en", 
    "TargetLanguageCode": "fr"
}

SyntaxError: EOL while scanning string literal (<ipython-input-36-211d8b150847>, line 2)

We can paste the body of text from Dostoevsky's short story into the **"Text"** key of the .json. 

For the **"SourceLanguageCode"**, put "auto" instead of "en". We can use "auto" because Translate can detect the language that is in the .json. The code "ru" can also be written instead of "auto" since we know that Dostoevsky's short story is written in the Russian language. 

For the **"TargetLanguageCode"** key, write "en" instead of "fr" because we wanted to translate his story into English, not French  

#### Another important note

When pasting the body of text into the .json, make sure to get rid of line breaks. If you do not do this, the code will not be readable. 

#### Let's Keep Going

So, the code for this particular example should look something like the below: 

In [ ]:
{
    "Text": "II. Мальчик у Христа на ёлке Но я романист, и, кажется, одну «историю» сам сочинил. Почему я пишу: «кажется», ведь я сам знаю наверно, что сочинил, но мне все мерещится, что это где-то и когда-то случилось, именно это случилось как раз накануне рождества, в каком-то огромном городе и в ужасный мороз. Мерещится мне, был в подвале мальчик, но еще очень маленький, лет шести или даже менее. Этот мальчик проснулся утром в сыром и холодном подвале. Одет он был в какой-то халатик и дрожал. Дыхание его вылетало белым паром, и он, сидя в углу на сундуке, от скуки нарочно пускал этот пар изо рта и забавлялся, смотря, как он вылетает. Но ему очень хотелось кушать. Он несколько раз с утра подходил к нарам, где на тонкой, как блин, подстилке и на каком-то узле под головой вместо подушки лежала больная мать его. Как она здесь очутилась? Должно быть, приехала с своим мальчиком из чужого города и вдруг захворала. Хозяйку углов захватили еще два дня тому в полицию; жильцы разбрелись, дело праздничное, а оставшийся один халатник уже целые сутки лежал мертво пьяный, не дождавшись и праздника. В другом углу комнаты стонала от ревматизма какая-то восьмидесятилетняя старушонка, жившая когдато и где-то в няньках, а теперь помиравшая одиноко, охая, брюзжа и ворча на мальчика, так что он уже стал бояться подходить к ее углу близко. Напиться-то он где-то достал в сенях, но корочки нигде не нашел и раз в десятый уже подходил разбудить свою маму. Жутко стало ему, наконец, в темноте: давно уже начался вечер, а огня не зажигали. Ощупав лицо мамы, он подивился, что она совсем не двигается и стала такая же холодная, как стена. «Очень уж здесь холодно», — подумал он, постоял немного, бессознательно забыв свою руку на плече покойницы, потом дохнул на свои пальчики, чтоб отогреть их, и вдруг, нашарив на нарах свой картузишко, потихоньку, ощупью, пошел из подвала. Он еще бы и раньше пошел, да все боялся вверху, на лестнице, большой собаки, которая выла весь день у соседских дверей. Но собаки уже не было, и он вдруг вышел на улицу.", 
    "SourceLanguageCode": "auto", 
    "TargetLanguageCode": "en"
}

If we take a look at the screenshot below, this is how the code will look like in the .json editor. 

![json-editor](./json-editor-screenshot)

Save that .json file to the disk as seen in the screenshot above. Save it as the name "translate.json". Then, upload that file to the folder you created earlier. One way to do this is to go to the JuptyerLab file section where your folder is located and click the "Upload" button. Your "translate.json" file should save in the folder. See the screenshot below for a visual. The "Upload" button is circled in red and the file is circled in blue.


![translate-json-visual](translate-visual) 

To check if the file is in your directory, use the `!ls` command. 

In [ ]:
!ls

To view the file, use the `!cat` command. 

In [ ]:
!cat translate.json

#### Now for the magic... 

Run the following command from the Translate ML service to actually translate the .json. The CLI code requires that you input the region where you are located which in our case is *us-east-1*. The translate.json will be converted into another .json called translated.json which will appear in your directory. This new file will contain the translated text. 

In [ ]:
!aws translate translate-text \
            --region us-east-1 \
            --cli-input-json file://translate.json > translated.json

In [ ]:
You can use the following command to view the translated text: 

In [ ]:
!cat translated.json

....and that's all there is to this method. Now onto the Python SDK method.... 

### Python SDK Method

We can use the same notebook we used before to initiate this method. First, import the package for the Python SDK, `boto3`.

In [ ]:
import boto3

Next, create a `client` instance of the client object in the `boto3` package for `translate`. We can now make requests from this service. 

In [ ]:
translate = boto3.client(service_name='translate', use_ssl=True)

Before we actually use the service, we will create a Python variable that contains our Dostoevsky short story within it as a string. 

In [ ]:
dost = "II. Мальчик у Христа на ёлке Но я романист, и, кажется, одну «историю» сам сочинил. Почему я пишу: «кажется», ведь я сам знаю наверно, что сочинил, но мне все мерещится, что это где-то и когда-то случилось, именно это случилось как раз накануне рождества, в каком-то огромном городе и в ужасный мороз. Мерещится мне, был в подвале мальчик, но еще очень маленький, лет шести или даже менее. Этот мальчик проснулся утром в сыром и холодном подвале. Одет он был в какой-то халатик и дрожал. Дыхание его вылетало белым паром, и он, сидя в углу на сундуке, от скуки нарочно пускал этот пар изо рта и забавлялся, смотря, как он вылетает. Но ему очень хотелось кушать. Он несколько раз с утра подходил к нарам, где на тонкой, как блин, подстилке и на каком-то узле под головой вместо подушки лежала больная мать его. Как она здесь очутилась? Должно быть, приехала с своим мальчиком из чужого города и вдруг захворала. Хозяйку углов захватили еще два дня тому в полицию; жильцы разбрелись, дело праздничное, а оставшийся один халатник уже целые сутки лежал мертво пьяный, не дождавшись и праздника. В другом углу комнаты стонала от ревматизма какая-то восьмидесятилетняя старушонка, жившая когдато и где-то в няньках, а теперь помиравшая одиноко, охая, брюзжа и ворча на мальчика, так что он уже стал бояться подходить к ее углу близко. Напиться-то он где-то достал в сенях, но корочки нигде не нашел и раз в десятый уже подходил разбудить свою маму. Жутко стало ему, наконец, в темноте: давно уже начался вечер, а огня не зажигали. Ощупав лицо мамы, он подивился, что она совсем не двигается и стала такая же холодная, как стена. «Очень уж здесь холодно», — подумал он, постоял немного, бессознательно забыв свою руку на плече покойницы, потом дохнул на свои пальчики, чтоб отогреть их, и вдруг, нашарив на нарах свой картузишко, потихоньку, ощупью, пошел из подвала. Он еще бы и раньше пошел, да все боялся вверху, на лестнице, большой собаки, которая выла весь день у соседских дверей. Но собаки уже не было, и он вдруг вышел на улицу."

We will then use the `translate_text` method from this client to translate the text. Notice how similar the code is to the AWS CLI method. 

In [ ]:
result2 = translate.translate_text(Text= dyost, SourceLanguageCode='auto', TargetLanguageCode= 'en')

The code below prints the results: 

In [ ]:
print(result2)

....and that's all for this method. Below I will talk about...

## Interesting ML Services, Potential Use Cases, and More!

Some more commentary on my process below. 

### More on the Translate Service 

The Translate service can do so much more than the above. Besides translating documents in real time, it can dictate translated documents in the native voice of the language using the ML service [**Amazon Polly**](https://docs.aws.amazon.com/polly/latest/dg/what-is.html), translate an entire chat channel on a website or application, or translate a large batch of documents using an Asynchronous Batch Process which utlizies S3 buckets. Overall, this service is very useful in processing large amounts of language data or for NLP purposes. 

### Interesting ML Services I Couldn't Use

When looking at the ML services, I was interested in [**Amazon Healthlake**](https://docs.aws.amazon.com/healthlake/latest/devguide/what-is-amazon-health-lake.html) which can be used 

>to store, transform, query, and analyze data in the AWS Cloud. Using the HealthLake integrated medical natural language processing (NLP) capabilities, you can analyze unstructured clinical text from diverse sources. HealthLake transforms unstructured data using natural language processing models, and provides powerful query and search capabilities. You can use HealthLake to organize, index, and structure patient information in a secure, compliant, and auditable manner.

I was also interested in [**Amazon Fraud Detector**](https://docs.aws.amazon.com/frauddetector/latest/ug/what-is-frauddetector.html) which 

>enables you to build fraud-detection models. You don't need ML experience to start using Amazon Fraud Detector, you only need to provide your company’s historical fraud data. Amazon Fraud Detector uses this data to automatically train, test, and deploy a customized fraud detection model.

However, I could not use either of these services because I do not personally have lots of health data at my disposal nor do I have historical fraud data from my company. That being said, it would be cool to explore these data sources in the future. Which brings me to...


### Potential Use Cases 

These two services have incredible use cases. 

Healthlake can be used to manage large stores of health data and conduct analytics for better patient care in the future and increased efficiency in medical centeres. 

Fraud Detector can improve a company's security like never before. It can predict how fraud will occur by training a model on historical fraud data. 